### Example for rectangle with ALPHA

```
from tkinter import *
from PIL import Image, ImageTk

root = Tk()

images = []  # to hold the newly created image

def create_rectangle(x1, y1, x2, y2, **kwargs):
    if 'alpha' in kwargs:
        alpha = int(kwargs.pop('alpha') * 255)
        fill = kwargs.pop('fill')
        fill = root.winfo_rgb(fill) + (alpha,)
        image = Image.new('RGBA', (x2-x1, y2-y1), fill)
        images.append(ImageTk.PhotoImage(image))
        canvas.create_image(x1, y1, image=images[-1], anchor='nw')
    canvas.create_rectangle(x1, y1, x2, y2, **kwargs)

canvas = Canvas(width=300, height=200)
canvas.pack()

create_rectangle(10, 10, 200, 100, fill='blue')
create_rectangle(50, 50, 250, 150, fill='green', alpha=.5)
create_rectangle(80, 80, 150, 120, fill='#800000', alpha=.8)

root.mainloop()
```

In [278]:
from tkinter import *
import numpy as np
import copy


In [294]:
#Coordinates go from 0 to 8
#Numbers go from 1 to 9, 0 means empty cell
#"pm": Pencilmark

class Grid:
    def __init__(self, orig = None):
        self.grid       = np.zeros( (9,9) ).astype(np.int8)
        
        #this is handled by the class, though user can change it
        self.options    = np.zeros( (9,9,10) ).astype(np.int8) #boolean for each option
        self.options_nr = np.zeros( (9,9) ).astype(np.int8)    #nr of options
        self.options_sh = np.ones( (9,9) ).astype(np.int8)    #boolean if options are shown in a cell (i.e. centerpm)

        #this is handled by the user
        self.toppm      = np.zeros( (9,9,10) ).astype(np.int8) #boolean for each option

        self.init_grid()
        
    def init_grid(self):
        for i in range(9):
            for j in range(9):
                self.grid[i,j]       = 0    #The cell is empty at the beginning
                self.options_nr[i,j] = 9    #All options are available at the beginning -> #(1..9)=9
                self.options_sh[i,j] = 0    #Not showing the options at the beginning
                self.options[i,j,0]  = 0    #0 is never an option
                self.toppm[i,j,0]    = 0    #0 is never a top pm
                for k in range(1,10):
                    self.options[i,j,k] = 1 #All options are available at the beginning
                    self.toppm[i,j,k]   = 0 #There is no top pm at the beginning
                    
    def get_digit(self,i,j):
        return self.grid[i,j]

    def get_centerpm(self,i,j):
        res = ''
        if self.options_sh[i,j]:
            for k in range(1,10):
                if self.options[i,j,k]==1:
                    res += str(k)
        return res

    def is_centerpm(self,i,j,digit):
        return self.options[i,j,digit]

    def get_toppm(self,i,j):
        res = ''
        for k in range(1,10):
            if self.toppm[i,j,k]==1:
                res += str(k)
        return res

    def is_toppm(self,i,j,digit):
        return self.toppm[i,j,digit]
    
    def add_digit(self, i, j, digit):
        if self.options[i,j,digit]==0:
            return False
        self.grid[i,j]=digit
        self.update_options()
        return True
    
    def change_centerpm(self,i,j,digit):
        #If not visible -> no change
        if not self.options_sh[i,j]:
            return False
        if self.options[i,j,digit]==1:
            self.options[i,j,digit] = 0
            self.options_nr[i,j] -= 1
            self.toppm[i,j,digit]=0
            return True
        else:
            #Undeleting option is not allowed, because it loses information (w.r.t. toppm)
            return False
            
            #valid=True
            #for k in range(9):
            #    if k!=i:
            #        if self.grid[k,j]==digit:
            #            valid = False
            #    if k!=j:
            #        if self.grid[i,k]==digit:
            #            valid = False
            #for ii in range(3):
            #    for jj in range(3):
            #        ci = i-i%3+ii
            #        cj = j-j%3+jj
            #        if ci!=i or cj!=j:
            #            if self.grid[ci,cj]==digit:
            #                valid = False
            #if valid:
            #    self.options[i,j,digit]=1
            #    self.options_nr[i,j] += 1
            #return valid

    def is_centerpm_shown(self,i,j):
        return self.options_sh[i,j]
            
    def toggle_centerpm_shown(self,i,j):
        self.options_sh[i,j] = 1 - self.options_sh[i,j]           
            
    def change_toppm(self,i,j,digit):
        if self.toppm[i,j,digit]==1:
            self.toppm[i,j,digit] = 0
            return True
        else:
            valid=self.options[i,j,digit]
            if valid:
                self.toppm[i,j,digit]=1
            return valid
    
    def update_options(self):
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]!=0:                    
                    d = self.grid[i,j]
                    for dd in range(1,10):
                        self.toppm[i,j,dd]=0
                        if d!=dd:
                            if self.options[i,j,dd]==1:
                                self.options[i,j,dd]=0                        
                                self.options_nr[i,j]-=1                            
                    for k in range(9):
                        self.toppm[i,k,d]=0
                        self.toppm[k,j,d]=0
                        if k!=j:
                            if self.options[i,k,d] == 1:
                                self.options[i,k,d] = 0
                                self.options_nr[i,k] -= 1                                            
                        if k!=i:
                            if self.options[k,j,d] == 1:
                                self.options[k,j,d] = 0
                                self.options_nr[k,j] -= 1
                    i0 = i - i%3
                    j0 = j - j%3
                    for ii in range(3):
                        for jj in range(3):
                            self.toppm[i0+ii,j0+jj,d]=0
                            if i0+ii!=i or j0+jj!=j:
                                if self.options[i0+ii,j0+jj,d]==1:
                                    self.options[i0+ii,j0+jj,d]=0
                                    self.options_nr[i0+ii,j0+jj]-=1

    def is_broken(self):
        res = False
        for i in range(9):
            for j in range(9):
                if self.options_nr[i,j]==0:
                    res = True
        return res

    def is_solved(self):
        res = True
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]==0:
                    res = False
        return res

        

In [298]:
class GridList:
    def __init__(self):    
        grid = Grid()
        self.gridlist = [grid]
        self.curri = 0 #Index of current grid
        
    def getcurr(self):
        res =  self.gridlist[self.curri]
        return res 
    
    def add_digit(self, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = newgrid.add_digit(i,j,digit)
        if success:
            self.gridlist = self.gridlist[0:self.curri+1]
            self.gridlist.append(newgrid)
            self.curri = len(self.gridlist)-1
    
    def change_centerpm(self, selected, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected: 
            nr = 0
            for (ii,jj) in selected:
                if newgrid.is_centerpm(ii,jj,digit):
                    nr += 1
            for (ii,jj) in selected:
                if not newgrid.is_centerpm_shown(ii,jj):
                    newgrid.toggle_centerpm_shown(ii,jj)
                if nr>0:
                    if newgrid.is_centerpm(ii,jj,digit):
                        success |= newgrid.change_centerpm(ii,jj,digit)                                    
                else:
                    if not newgrid.is_centerpm(ii,jj,digit):
                        success |= newgrid.change_centerpm(ii,jj,digit)                    
        else: #no selection: using currently framed cell
            success = newgrid.change_centerpm(i,j,digit)
        if success:
            self.gridlist = self.gridlist[0:self.curri+1]
            self.gridlist.append(newgrid)
            self.curri = len(self.gridlist)-1
        
            

    def change_toppm(self, selected, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected: 
            nr=0
            for (ii,jj) in selected:
                if newgrid.is_toppm(ii,jj,digit):
                    nr += 1
            for (ii,jj) in selected:
                if nr>0:
                    if newgrid.is_toppm(ii,jj,digit):
                        success |= newgrid.change_toppm(ii,jj,digit)                    
                else:
                    if not newgrid.is_toppm(ii,jj,digit):
                        success |= newgrid.change_toppm(ii,jj,digit)                    
        else:
            success = newgrid.change_toppm(i,j,digit)
        if success:
            self.gridlist = self.gridlist[0:self.curri+1]
            self.gridlist.append(newgrid)
            self.curri = len(self.gridlist)-1
           
    def centerpm_onoff(self, selected, i, j):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        if selected: 
            nr=0
            for (ii,jj) in selected:
                if newgrid.is_centerpm_shown(ii,jj):
                    nr += 1
            for (ii,jj) in selected:
                if nr>0:
                    if newgrid.is_centerpm_shown(ii,jj):
                        newgrid.toggle_centerpm_shown(ii,jj)
                else:
                    if not newgrid.is_centerpm_shown(ii,jj):
                        newgrid.toggle_centerpm_shown(ii,jj)
        else:
            newgrid.toggle_centerpm_shown(i,j)
        self.gridlist = self.gridlist[0:self.curri+1]
        self.gridlist.append(newgrid)
        self.curri = len(self.gridlist)-1
            
    def undo(self):
        if self.curri>0:
            self.curri -= 1
    
    def redo(self):
        if self.curri<len(self.gridlist)-1:
            self.curri += 1
    
    def is_solved(self):
        return self.gridlist[self.curri].is_solved()
        
    def is_broken(self):
        return self.gridlist[self.curri].is_broken()
        
        
        
        

In [299]:
VP=HP=65
VW=HW=65
TOPPM_PADDING = HP//3
PM_ADJ = 2
LW1 = 2
LW2 = 3
FRAMEWIDTH=3

FONT_L      = "Mono"
FONTSIZE_L  = "9"
FONT        = "Mono"
FONTSIZE    = "40"
FONT_PM     = "Mono"
FONTSIZE_PM = "10"

SOLVEDCOLOR = 'green'
BROKENCOLOR = 'red'
BGCOLOR     = 'white'
LINECOLOR   = 'black'
FONTCOLOR   = 'darkblue'
FRAMECOLOR  = 'red'
SELCOLOR    = 'yellow'
SELALPHA    = 0.5

class UI:
    def __init__(self):
        self.curri = 0
        self.currj = 0
        self.selected = set()
        
        self.framelines = []
    
        self.gridlist = GridList()

        self.root = Tk()
        self.canvas = Canvas(self.root, width=2*VP+9*VW+1, height=2*HP+9*HW+1, background=BGCOLOR)
        self.Draw(delete=False)

        self.canvas.pack()
        
        #Key bindings
        self.canvas.bind("<Up>",    self.key_arrow)
        self.canvas.bind("<Down>",  self.key_arrow)
        self.canvas.bind("<Left>",  self.key_arrow)
        self.canvas.bind("<Right>", self.key_arrow)
        
        self.canvas.bind("<Control-z>", self.undo)
        self.canvas.bind("u",           self.undo)
        self.canvas.bind("<Control-Z>", self.redo)
        self.canvas.bind("r",           self.redo)
        self.canvas.bind("a",           self.select_all)
        self.canvas.bind("A",           self.select_all)
        self.canvas.bind("<Control-a>", self.deselect_all)
        self.canvas.bind("<Control-A>", self.deselect_all)
        self.canvas.bind("c",           self.centerpm_onoff)
        
        #All other keys
        self.canvas.bind("<Key>",   self.key)
        
        #Mouse bindings
        self.canvas.bind("<Button-1>", self.leftclick)
        self.canvas.bind("<Shift-Button-1>", self.leftclick)
        self.canvas.bind("<Control-Button-1>", self.leftclick)
                        
        self.canvas.focus_set()
        
    def mainloop(self):
        self.root.mainloop()        
       
    
    ##########################
    # Drawing methods
    
    def DrawBase(self, canvas):
        color = LINECOLOR
        if self.gridlist.is_solved():
            color = SOLVEDCOLOR
        elif self.gridlist.is_broken():
            color = BROKENCOLOR
        for i in range(10):
            w=LW1
            if i%3==0:
                w=LW2
            canvas.create_line(VP, HP+HW*i, VP+9*VW, HP+HW*i, fill=color, width=w)
        for i in range(10):
            w=LW1
            if i%3==0:
                w=LW2
            canvas.create_line(VP+VW*i, HP, VP+VW*i, HP+HW*9, fill=color, width=w)
        #GridList info-ss
        canvas.create_text(1,1,anchor = "nw",
                           fill=FONTCOLOR,
                           font=FONT_L+" "+FONTSIZE_L,
                           text=str(self.gridlist.curri+1)+"/"+str(len(self.gridlist.gridlist)))        
    
    def DrawSelection(self, canvas):
        for (i,j) in self.selected:
            t=VP+VW*i+LW1
            l=HP+HW*j+LW1
            b=VP+VW*(i+1)-LW1
            r=HP+HW*(j+1)-LW1
            canvas.create_rectangle(t, l, b, r, fill=SELCOLOR, outline='')

    def DrawState(self, canvas, grid):
        for i in range(9):
            for j in range(9):
                digit = grid.get_digit(i,j)
                cpm = grid.get_centerpm(i,j)
                tpm = grid.get_toppm(i,j)
                if digit!=0:                
                    canvas.create_text(VP+VW*i+VW/2,HP+HW*j+HW/2,
                                       anchor = "center",
                                       fill=FONTCOLOR,
                                       font=FONT+" "+FONTSIZE,
                                       text=str(digit))
                else:
                    if cpm!='':                
                        canvas.create_text(VP+VW*i+VW/2-PM_ADJ,HP+HW*j+HW/2,
                                           anchor = "center",
                                           fill=FONTCOLOR,
                                           font=FONT_PM+" "+FONTSIZE_PM,
                                           text=cpm)
                    if tpm!='':
                        canvas.create_text(VP+VW*i+VW/2-PM_ADJ,HP+HW*j+HW/2-TOPPM_PADDING,
                                           anchor = "center",
                                           fill=FONTCOLOR,
                                           font=FONT_PM+" "+FONTSIZE_PM+ " italic",
                                           text=tpm)
    def DrawFrame(self, canvas, i, j):
        if self.framelines:
            for e in self.framelines:
                canvas.delete(e)
                
        e = canvas.create_rectangle( VP+VW*i, HP+HW*j, 
                                     VP+VW*(i+1), HP+HW*(j+1), 
                                     outline=FRAMECOLOR, width=FRAMEWIDTH)
        self.framelines.append(e)
            
    def Draw(self, delete=True):
        if delete:
            self.canvas.delete('all')
        self.DrawBase(self.canvas)
        self.DrawSelection(self.canvas)
        self.DrawState(self.canvas, self.gridlist.getcurr())
        self.DrawFrame(self.canvas, self.curri, self.currj)

        
    ##########################
    # Event handler methods    
    
    def key_arrow(self, event):
        oldi = self.curri
        oldj = self.currj
        if event.keycode==38:    #up
            self.currj-=1+9
            self.currj%=9
        elif event.keycode==40:  #down
            self.currj+=1
            self.currj%=9
        elif event.keycode==37:  #left
            self.curri-=1+9
            self.curri%=9
        elif event.keycode==39:  #right
            self.curri+=1
            self.curri%=9
        if event.state & 0x0001: #shift
            if not (oldi,oldj) in self.selected:
                self.selected.add( (oldi,oldj) )
            if not (self.curri,self.currj) in self.selected:
                self.selected.add( (self.curri,self.currj) )
            self.Draw()
        elif event.state & 0x0004: #control
            if (oldi,oldj) in self.selected:
                self.selected.remove( (oldi,oldj) )
            if (self.curri,self.currj) in self.selected:
                self.selected.remove( (self.curri,self.currj) )
            self.Draw()
        else:
            self.DrawFrame(self.canvas, self.curri, self.currj)


    def key(self, event):  
        # Numbers from 1 to 9
        if event.keycode>=49 and event.keycode<=57:
            digit = event.keycode - 48
            if event.state & 0x0004: #control
                self.gridlist.change_toppm(self.selected, self.curri, self.currj, digit)
            elif event.state & 0x0001: #shift
                self.gridlist.change_centerpm(self.selected, self.curri, self.currj, digit)
            else: #control
                self.gridlist.add_digit(self.curri, self.currj, digit)            
        # Selecting current cell
        elif event.char==' ':
            if (self.curri,self.currj) in self.selected:
                self.selected.remove( (self.curri,self.currj) )
            else:
                self.selected.add( (self.curri,self.currj) )
        self.Draw()
        
    def undo(self, event):
        self.gridlist.undo()
        self.Draw()
        
    def redo(self, event):
        self.gridlist.redo()
        self.Draw()

    def select_all(self, event):
        self.selected = set()
        for i in range(9):
            for j in range(9):
                self.selected.add((i,j))
        self.Draw()

    def deselect_all(self, event):
        self.selected = set()
        self.Draw()
        
    def centerpm_onoff(self, event):
        self.gridlist.centerpm_onoff(self.selected, self.curri, self.currj)
        self.Draw()
        
    def leftclick(self, event):
        x = event.x-HP
        y = event.y-VP
        i = x//HW
        j = y//VW
        if i>=0 and i<9 and j>=0 and j<9:
            self.curri = i
            self.currj = j
            if event.state & 0x0004: #control
                if (self.curri,self.currj) in self.selected:
                    self.selected.remove( (self.curri,self.currj) )
                self.Draw()                                            
            elif event.state & 0x0001: #shift
                if not (self.curri,self.currj) in self.selected:
                    self.selected.add( (self.curri,self.currj) )            
                self.Draw()                            
            else: # neither control nor shift
                self.DrawFrame(self.canvas, self.curri, self.currj)            


        

In [300]:
ui = UI()
ui.mainloop()